###  Created by Luis Alejandro (alejand@umich.edu)

In [1]:
import os
import tensorflow as tf
from aircraft_models import AirMultinomialTemporalCNN
from tfrecord_dataset import feature_description

In [2]:
# Constants
AUTOTUNE = tf.data.experimental.AUTOTUNE
NUMBER_MFCC = 128

In [3]:
# Dataset location
train_file = 'exports/2020-02-07 01-09-35/train.tfrecord'
test_file = 'exports/2020-02-07 01-09-35/test.tfrecord'

In [4]:
# Parses observation from proto format
def parse_observation(example):
    observation = tf.io.parse_single_example(example, feature_description)
    observation['spec'] = tf.reshape(observation['spec'],(NUMBER_MFCC,-1))
    return observation
    
# Converts into correct format for training (input,output) = (spec,label)
def transform_observation(data):
    classes = [b'A320-2xx (CFM56-5)',b'B737-7xx (CF56-7B22-)',b'ERJ190 (CF34-10E)',b'B737-8xx (CF56-7B22+)']
    output = 0
    for i, label in enumerate(classes):
        if data['label'] == label:
            output = i
    return tf.transpose(data['spec']), output

In [5]:
# Creates training data pipeline
train_dataset = tf.data.TFRecordDataset([train_file])
train_dataset = train_dataset.map(parse_observation,num_parallel_calls=AUTOTUNE)
observation = next(iter(train_dataset))
_,time = observation['spec'].shape
train_dataset = train_dataset.map(transform_observation,num_parallel_calls=AUTOTUNE).cache().batch(128).prefetch(1)

In [6]:
# Creates test data pipeline
test_dataset = tf.data.TFRecordDataset([test_file])
test_dataset = test_dataset.map(parse_observation,num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(transform_observation,num_parallel_calls=AUTOTUNE).cache().batch(128).prefetch(1)

In [7]:
# Building dataset and model
builder = AirMultinomialTemporalCNN(categories=4)
model = builder.build_model((time,NUMBER_MFCC))
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 401, 128)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, 51, 16)            32784     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 51, 16)            784       
_________________________________________________________________
global_max_pooling1d (Global (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 33,636
Trainable params: 33,636
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Training model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
model.fit(train_dataset,epochs=30,validation_data=test_dataset)

Epoch 1/30
7/7 [==============================] - 5s 666ms/step - loss: 2.8701 - accuracy: 0.2089 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/30
7/7 [==============================] - 2s 272ms/step - loss: 2.1011 - accuracy: 0.3333 - val_loss: 1.8930 - val_accuracy: 0.3824
Epoch 3/30
7/7 [==============================] - 3s 390ms/step - loss: 1.8780 - accuracy: 0.4212 - val_loss: 1.8175 - val_accuracy: 0.5000
Epoch 4/30
7/7 [==============================] - 2s 260ms/step - loss: 1.7700 - accuracy: 0.4943 - val_loss: 1.7629 - val_accuracy: 0.4706
Epoch 5/30
7/7 [==============================] - 2s 258ms/step - loss: 1.6823 - accuracy: 0.5479 - val_loss: 1.7054 - val_accuracy: 0.4706
Epoch 6/30
7/7 [==============================] - 2s 254ms/step - loss: 1.5932 - accuracy: 0.5879 - val_loss: 1.6616 - val_accuracy: 0.5000
Epoch 7/30
7/7 [==============================] - 2s 254ms/step - loss: 1.5199 - accuracy: 0.6370 - val_loss: 1.6202 - val_accuracy: 0.4706
Epoch 8/30
7